# <span style="color:darkred">Additive Schwarz  using multiple strategies</span>.

<img align="left" src="Schwarz.jpg" width=120 height=120 />

### [Simon Tavener](https://www.math.colostate.edu/~tavener/) and Andre Leautaud

#### Full text for this notebook can be found at the following [link](https://www.math.colostate.edu/~tavener/andre/DD_multiphysics.pdf)

This notebook illustrates three ways of solving a system using Additive Schwarz. It first creates partitions and necessary matrices, then steps through the forms of iteration. It demonstrates C Iteration (each subdomain independently), D Iteration (subdomains calculated in parallel) and  U Iteration (computes all iterations as one matrix solve). In each, we keep track of total error, iteration error, and discretization error (true and estimates). If the estimates are good, and the system was solved, the effectivty ratio will be 1. It can read all parameters and the linear system from a csv. It can also create a csv. This csv is compatible with Multiplicative Schwarz and the MatLab Live files.

#### Set path and parameters

In [79]:
# load necessary packages
import numpy as np
import math
%run DD_Utilities.ipynb
%run AS_Utilities.ipynb
iprint= 4 # amount of output desired, 0, 2, 4, or 6

In [80]:
iparam = 0

In [81]:
if iparam == 0:
    n= 10  # size of system
    p= 4   # number of subomains
    b= [1,2,4,7]  # indices of starts of subdomains (1 indexed), should be in ascending order.
    e= [3,4,8,10] # indices of ends of subdomains (1 indexed), need not be in ascending order. 
                  # Setting e to an integer will make each subdomain overlap the next by that number.

    K= 3   # number of iterations, should be less than 500
    alpha= 0.5 # alpha is only used in multiplicative schwarz. Including it here makes the file i/o compatible.
    mpert= 0.02

In [82]:
# This example shows how to create domains that each overlap by 1 column
#small system
if iparam == 1:
    n= 8
    p= 3 
    b= [1,3,5]
    e= 1 # causes each subdomain to overlap the previous by 1
    K= 3
    alpha= 0.5
    mpert= 0.02

In [83]:
# some subdomains embedded in others
if iparam == 2:
    n= 10
    p= 5
    b= [1,1,5,5,7]
    e= [2,4,6,8,10] 
    K= 6
    alpha= 0.5
    mpert= 0.02

In [84]:
# overlap does not connect all subdomains
if iparam == 3:
    n= 9
    p= 4 
    b= [1,3,6,8]
    e= [2,5,7,9]
    K= 3
    alpha= 0.5
    mpert= 0.02

In [85]:
if iparam == 4:
    n= 12
    p= 4 
    b= [1,2,5,9]
    e= [3,7,10,12]
    K= 3
    alpha= 0.5
    mpert= 0.02

In [86]:
print(p,b,e)
nn,nb,ne,index= fix_domains(n,p,b,e,iprint)

4 [1, 2, 4, 7] [3, 4, 8, 10]


### Establish and solve forward system, $Mx=y$

### Establish subdomains, partitions, number of iterations and random numerical error.  Read and/or save as appropriate

In [87]:
# Read and write capability. Usually only use one of sread and swrite, but it is acceptable to set both to True.
sread= False   #Read system of equations from file_name. The solver will then ignore all of the parameters set above.
swrite= True    #Write system to file_name file
file_name= 'iparam' + str(iparam) + '.csv'

M,x,y,xstore= Linear_system(n,sread,file_name)
wpert= Make_wpert(n,p,K,sread,file_name) #Read or create wpert (depending on sread)

if iprint >= 4:
    print(M)
    print('y')
    print(y)
    print('x')
    print(x)

if swrite:
    save_to_csv(n,p,K,alpha,nb,nn,ne,M,y,mpert,wpert,iparam)

[[1.04170220e+01 7.20324493e-01 1.14374817e-04 3.02332573e-01
  1.46755891e-01 9.23385948e-02 1.86260211e-01 3.45560727e-01
  3.96767474e-01 5.38816734e-01]
 [4.19194514e-01 1.06852195e+01 2.04452250e-01 8.78117436e-01
  2.73875932e-02 6.70467510e-01 4.17304802e-01 5.58689828e-01
  1.40386939e-01 1.98101489e-01]
 [8.00744569e-01 9.68261576e-01 1.03134242e+01 6.92322616e-01
  8.76389152e-01 8.94606664e-01 8.50442114e-02 3.90547832e-02
  1.69830420e-01 8.78142503e-01]
 [9.83468338e-02 4.21107625e-01 9.57889530e-01 1.05331653e+01
  6.91877114e-01 3.15515631e-01 6.86500928e-01 8.34625672e-01
  1.82882773e-02 7.50144315e-01]
 [9.88861089e-01 7.48165654e-01 2.80443992e-01 7.89279328e-01
  1.01032260e+01 4.47893526e-01 9.08595503e-01 2.93614148e-01
  2.87775339e-01 1.30028572e-01]
 [1.93669579e-02 6.78835533e-01 2.11628116e-01 2.65546659e-01
  4.91573159e-01 1.00533625e+01 5.74117605e-01 1.46728575e-01
  5.89305537e-01 6.99758360e-01]
 [1.02334429e-01 4.14055988e-01 6.94400158e-01 4.14179270e

### Iterative strategy
$u^{\{k\}}= u^{\{k-1\}} + Br = u^{\{k-1\}} + B(f-Au^{\{k-1\}})$, $\quad k=1, \dots, K,$ where $B$ is an approximation to $A^{-1}$

<ins>Additive<ins>
$A_i= R_i A R_i^\top \in \mathbb{R}^{m_i \times m_i} \\
B_i= R_i^\top A_i^{-1} R_i  = R_i^\top ( R_i A R_i^\top)^{-1} R_i  \in \mathbb{R}^{n \times n} \\
C_i= B_i A   \in \mathbb{R}^{n \times n} \\
f_i= B_i f   \in \mathbb{R}^{n}
$

In [88]:
#Construct domains, R,A,B,C, and f
R= Rmatrices(n,p,nn,index,iprint)
A= Amatrices(M,R,n,p,nn,iprint)
B= Bmatrices(A,R,n,p,nn,iprint)
C= Cmatrices(M,B,n,p,iprint) 
f= fvector(B,y,n,p,iprint)

Print f vector 
i= 0
[0.02812641 0.0467045  0.07933329 0.         0.         0.
 0.         0.         0.         0.        ]
i= 1
[0.         0.04575664 0.07993995 0.02481948 0.         0.
 0.         0.         0.         0.        ]
i= 2
[ 0.          0.          0.          0.02043265  0.08395934  0.0565028
 -0.00663193  0.08469071  0.          0.        ]
i= 3
[ 0.          0.          0.          0.          0.          0.
 -0.01206386  0.08072284  0.05269031  0.09058564]


####  $K$ iterations computing subdomain contributions independently [C iteration]
$
u^{\{k\}}= u^{\{k-1\}} + \alpha \sum_{i=1}^p B_i(f-Au^{\{k-1\}}) = u^{\{k-1\}} + \alpha \sum_{i=1}^p ( f_i-C_iu^{\{k-1\}}) 
$

In [89]:
#Perform C iteration
vstore= Citeration_additive(C,f,alpha,n,p,K,iprint)

Outer iteration 0
[[ 0.01406321  0.04623057  0.07963662  0.02262606  0.04197967  0.0282514
  -0.0093479   0.08270677  0.02634515  0.04529282]]
Outer iteration 1
[[ 0.01767509  0.03881777  0.06914326  0.01140214  0.05996826  0.038255
  -0.01861866  0.07756918  0.03334546  0.06282098]]
Outer iteration 2
[[ 0.01903506  0.03889105  0.06554565  0.01078518  0.06905644  0.04282024
  -0.01849901  0.07660322  0.03678929  0.07193951]]


In [90]:
if iprint >= 4:
    print('vstore')
    print(vstore)

vstore
[[ 0.01406321]
 [ 0.04623057]
 [ 0.07963662]
 [ 0.02262606]
 [ 0.04197967]
 [ 0.0282514 ]
 [-0.0093479 ]
 [ 0.08270677]
 [ 0.02634515]
 [ 0.04529282]
 [ 0.01767509]
 [ 0.03881777]
 [ 0.06914326]
 [ 0.01140214]
 [ 0.05996826]
 [ 0.038255  ]
 [-0.01861866]
 [ 0.07756918]
 [ 0.03334546]
 [ 0.06282098]
 [ 0.01903506]
 [ 0.03889105]
 [ 0.06554565]
 [ 0.01078518]
 [ 0.06905644]
 [ 0.04282024]
 [-0.01849901]
 [ 0.07660322]
 [ 0.03678929]
 [ 0.07193951]]


####  $K$ iterations combining $p$  (parallel) subdomain solves [D iteration]

<ins>Additive Schwarz<ins>
    
$u^{\{k\}}= D u^{\{k-1\}} + g, \qquad 
D = I-\alpha \sum_{i=1}^p C_i, \quad g= \alpha \sum_{i=1}^p f_i$


In [91]:
# Perform D iteration
#Construct D and g and solve
D= Dmatrix_additive(C,alpha,n,p,iprint)

dmatrix_additive:  matrix
[[ 5.00e-01 -0.00e+00 -0.00e+00 -1.17e-02 -7.00e-03 -2.30e-03 -7.60e-03
  -1.48e-02 -1.86e-02 -2.53e-02]
 [-1.89e-02  0.00e+00  0.00e+00 -4.01e-02  1.70e-03 -6.03e-02 -3.61e-02
  -4.86e-02 -1.21e-02 -1.34e-02]
 [-3.70e-02 -0.00e+00  0.00e+00 -2.89e-02 -8.26e-02 -8.02e-02 -2.10e-03
   4.50e-03 -1.39e-02 -7.98e-02]
 [ 1.30e-03 -1.56e-02 -4.28e-02  0.00e+00 -2.92e-02 -1.02e-02 -3.19e-02
  -3.90e-02  5.80e-03 -6.25e-02]
 [-4.77e-02 -3.29e-02 -7.50e-03 -0.00e+00  5.00e-01 -0.00e+00 -0.00e+00
  -0.00e+00 -8.90e-03  4.00e-04]
 [ 2.10e-03 -3.07e-02 -7.30e-03 -0.00e+00 -0.00e+00  5.00e-01 -0.00e+00
  -0.00e+00 -2.62e-02 -3.22e-02]
 [-2.10e-03 -3.17e-02 -5.91e-02 -1.54e-02  4.00e-04 -2.09e-02  0.00e+00
  -0.00e+00 -4.16e-02 -2.32e-02]
 [-8.25e-02 -2.50e-03 -7.00e-04 -3.51e-02 -1.72e-02 -2.50e-03 -0.00e+00
   0.00e+00 -3.22e-02 -3.01e-02]
 [-3.76e-02 -2.61e-02 -3.29e-02 -1.12e-02 -1.00e-02 -4.04e-02  0.00e+00
   0.00e+00  5.00e-01  0.00e+00]
 [-3.20e-03 -4.34e-02 -1.87e-

In [92]:
g= gvector_additive(f,alpha,n,p,iprint)

Print g vector
[[ 0.01406321]
 [ 0.04623057]
 [ 0.07963662]
 [ 0.02262606]
 [ 0.04197967]
 [ 0.0282514 ]
 [-0.0093479 ]
 [ 0.08270677]
 [ 0.02634515]
 [ 0.04529282]]


In [93]:
# print('Perform D iteration \n')
wstore= Diteration_additive(D,g,n,p,K,iprint)

Diteration_additive: w at iteration  0
[[ 0.03254028]
 [-0.0932145 ]
 [-0.13836287]
 [-0.12724521]
 [ 0.17702969]
 [ 0.42700268]
 [-0.16452912]
 [-0.0167966 ]
 [ 0.22468827]
 [ 0.44874272]]
Diteration_additive: w at iteration  1
[[ 0.01553034]
 [ 0.02328173]
 [-0.00542358]
 [-0.00030726]
 [ 0.13121031]
 [ 0.22534342]
 [-0.02496223]
 [ 0.05994513]
 [ 0.12684588]
 [ 0.27206851]]
Diteration_additive: w at iteration  2
[[ 0.01042788]
 [ 0.02538611]
 [ 0.02700841]
 [-0.00141403]
 [ 0.10509495]
 [ 0.12818648]
 [-0.02605409]
 [ 0.06627051]
 [ 0.07833757]
 [ 0.17588957]]
norm(D)=  1.151021e+00
Asymptotic solution: z= inv(I-D)@g
K=     3, norm(w-z)=  1.405649e-01


In [94]:
if iprint >= 2:
    print('wstore')
    print(wstore)

wstore
[[ 3.25402760e-02]
 [-9.32145038e-02]
 [-1.38362873e-01]
 [-1.27245210e-01]
 [ 1.77029688e-01]
 [ 4.27002676e-01]
 [-1.64529119e-01]
 [-1.67965977e-02]
 [ 2.24688275e-01]
 [ 4.48742721e-01]
 [ 1.55303354e-02]
 [ 2.32817269e-02]
 [-5.42358453e-03]
 [-3.07258782e-04]
 [ 1.31210305e-01]
 [ 2.25343423e-01]
 [-2.49622333e-02]
 [ 5.99451253e-02]
 [ 1.26845882e-01]
 [ 2.72068514e-01]
 [ 1.04278806e-02]
 [ 2.53861144e-02]
 [ 2.70084055e-02]
 [-1.41402983e-03]
 [ 1.05094947e-01]
 [ 1.28186476e-01]
 [-2.60540907e-02]
 [ 6.62705051e-02]
 [ 7.83375726e-02]
 [ 1.75889565e-01]]


#### Perform $K$ iterations as single matrix solve [U iteration]

$K=6$ iterations of multiplicative Schwarz can be written as the $nK$-dimensional system of equations

<ins>Additive Schwarz<ins>

$Uz=
\pmatrix{
 I & 0 & 0 & 0 & 0 & 0   \cr
 -D &I & 0 & 0 & 0 & 0   \cr
 0 & -D & I & 0 & 0 & 0   \cr
 0 & 0 & -D & I & 0 & 0   \cr
 0 & 0 & 0 & -D & I & 0   \cr
 0 & 0 & 0 & 0 & -D & I
}$
$\pmatrix{
v^{\{1\}}  \cr
v^{\{2\}}  \cr
v^{\{3\}}  \cr
v^{\{4\}}  \cr
v^{\{5\}}  \cr
v^{\{6\}}
}$
$\pmatrix{
g  \cr
g  \cr
g  \cr
g  \cr
g  \cr
g
}
=h$

In [95]:
# Perform U iteration
# Construct U and h
U= Umatrix_additive(D,n,K,iprint)
h= hvector_additive(g,n,K,iprint)

U matrix
[[ 1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   1.   0.   0.   0.   0.   0. 

In [96]:
# Solve using Uz= h
zstore= Uhsolve_additive(U,h,n,K,iprint)

Uhsolve_additive: z at iteration 0
[ 0.01406321  0.04623057  0.07963662  0.02262606  0.04197967  0.0282514
 -0.0093479   0.08270677  0.02634515  0.04529282]
Uhsolve_additive: z at iteration 1
[ 0.01767509  0.03881777  0.06914326  0.01140214  0.05996826  0.038255
 -0.01861866  0.07756918  0.03334546  0.06282098]
Uhsolve_additive: z at iteration 2
[ 0.01903506  0.03889105  0.06554565  0.01078518  0.06905644  0.04282024
 -0.01849901  0.07660322  0.03678929  0.07193951]


In [97]:
if iprint >= 4:
    print('zstore')
    print(zstore)

zstore
[[ 0.01406321]
 [ 0.04623057]
 [ 0.07963662]
 [ 0.02262606]
 [ 0.04197967]
 [ 0.0282514 ]
 [-0.0093479 ]
 [ 0.08270677]
 [ 0.02634515]
 [ 0.04529282]
 [ 0.01767509]
 [ 0.03881777]
 [ 0.06914326]
 [ 0.01140214]
 [ 0.05996826]
 [ 0.038255  ]
 [-0.01861866]
 [ 0.07756918]
 [ 0.03334546]
 [ 0.06282098]
 [ 0.01903506]
 [ 0.03889105]
 [ 0.06554565]
 [ 0.01078518]
 [ 0.06905644]
 [ 0.04282024]
 [-0.01849901]
 [ 0.07660322]
 [ 0.03678929]
 [ 0.07193951]]


#### Compare Solutions from all three methods

In [98]:
solution_compare(xstore,vstore,wstore,zstore,iprint)

norm(exact - iterative) solutions    =  1.386302e-02
Difference between vstore and wstore =  7.906816e-01
Difference between wstore and zstore =  7.906816e-01
Difference between vstore and zstore =  3.327777e-17


### Add discretization error and compute error estimate

#### C iteration with error

In [99]:
psi= np.ones((n,1))

# Solve system with "numerical error" for every solve
vglobal_store,vaglobal_store,rglobal_store= Citeration_additive_approx(C,f,alpha,n,p,K,mpert,wpert,iprint)

if iprint >= 4:
    print('vglobal_store')
    print(vglobal_store)
    print('vaglobal_store')
    print(vaglobal_store)
    print('rglobal_store')
    print(rglobal_store)    

Citeration_additive_approx: Outer iteration 0
Difference = 6.4823e-02
dmatrix_additive:  matrix
[[ 5.00e-01 -0.00e+00 -0.00e+00 -1.17e-02 -7.00e-03 -2.30e-03 -7.60e-03
  -1.48e-02 -1.86e-02 -2.53e-02]
 [-1.89e-02  0.00e+00  0.00e+00 -4.01e-02  1.70e-03 -6.03e-02 -3.61e-02
  -4.86e-02 -1.21e-02 -1.34e-02]
 [-3.70e-02 -0.00e+00  0.00e+00 -2.89e-02 -8.26e-02 -8.02e-02 -2.10e-03
   4.50e-03 -1.39e-02 -7.98e-02]
 [ 1.30e-03 -1.56e-02 -4.28e-02  0.00e+00 -2.92e-02 -1.02e-02 -3.19e-02
  -3.90e-02  5.80e-03 -6.25e-02]
 [-4.77e-02 -3.29e-02 -7.50e-03 -0.00e+00  5.00e-01 -0.00e+00 -0.00e+00
  -0.00e+00 -8.90e-03  4.00e-04]
 [ 2.10e-03 -3.07e-02 -7.30e-03 -0.00e+00 -0.00e+00  5.00e-01 -0.00e+00
  -0.00e+00 -2.62e-02 -3.22e-02]
 [-2.10e-03 -3.17e-02 -5.91e-02 -1.54e-02  4.00e-04 -2.09e-02  0.00e+00
  -0.00e+00 -4.16e-02 -2.32e-02]
 [-8.25e-02 -2.50e-03 -7.00e-04 -3.51e-02 -1.72e-02 -2.50e-03 -0.00e+00
   0.00e+00 -3.22e-02 -3.01e-02]
 [-3.76e-02 -2.61e-02 -3.29e-02 -1.12e-02 -1.00e-02 -4.04e-02  0

#### Calculate the discretization error and effectivity ratio

In [100]:
vglobal_store

array([[ 0.01406321],
       [ 0.04623057],
       [ 0.07963662],
       [ 0.02262606],
       [ 0.04197967],
       [ 0.0282514 ],
       [-0.0093479 ],
       [ 0.08270677],
       [ 0.02634515],
       [ 0.04529282],
       [ 0.01767509],
       [ 0.03881777],
       [ 0.06914326],
       [ 0.01140214],
       [ 0.05996826],
       [ 0.038255  ],
       [-0.01861866],
       [ 0.07756918],
       [ 0.03334546],
       [ 0.06282098],
       [ 0.01903506],
       [ 0.03889105],
       [ 0.06554565],
       [ 0.01078518],
       [ 0.06905644],
       [ 0.04282024],
       [-0.01849901],
       [ 0.07660322],
       [ 0.03678929],
       [ 0.07193951]])

In [101]:
# Adjoint solve
phiglobal_store= Citeration_additive_adjoint(C,psi,alpha,n,p,K,iprint)
if iprint >= 4:
    print('phiglobal_store')
    print(phiglobal_store.T)

# Error estimate= (r,\phi)
qoi_discretization_error_estimate= (rglobal_store.T @ phiglobal_store)[0,0]

if iprint >= 4:
    print(f'norm(r_store) = {np.linalg.norm(rglobal_store):13.6e}, norm(phi_store)= {np.linalg.norm(phiglobal_store):13.6e}')

# Compute error and error in QoI directly
vsoln= vglobal_store[n * (K-1): n * K, 0]
vasoln= vaglobal_store[n * K: n * (K+1), 0]
verror= vsoln.T-vasoln
qoi_discretization_error= (psi.T @ verror.T)[0]

# Calculate effectivity ratio
effectivity_ratio_disc= qoi_discretization_error_estimate/ qoi_discretization_error
print(f'QoI_discretization_error          = {qoi_discretization_error:13.6e}')
print(f'QoI_discretization_error_estimate = {qoi_discretization_error_estimate:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_disc:13.3f}')

10 4 3
Citeration_additive_adjoint: k= 2, i= 3
[0.16179776 0.16920738 0.16141054 0.17147101 0.09042594 0.14617614
 1.         1.         1.         1.        ]
Citeration_additive_adjoint: k= 2, i= 2
[0.17646575 0.196746   0.17649974 1.         1.         1.
 1.         1.         0.20622522 0.23195479]
Citeration_additive_adjoint: k= 2, i= 1
[0.1128471  1.         1.         1.         0.13604672 0.15965584
 0.09834497 0.11815382 0.0279641  0.15506265]
Citeration_additive_adjoint: k= 2, i= 0
[1.         1.         1.         0.16141379 0.09831205 0.146318
 0.05707609 0.0775008  0.06123098 0.14550909]
Citeration_additive_adjoint: k= 1, i= 3
[ 0.02011835  0.03632593  0.02741914  0.009825    0.01232904  0.02903463
 -0.07771053 -0.09782731  0.35228986  0.23373673]
Citeration_additive_adjoint: k= 1, i= 2
[ 0.02311214  0.03075559 -0.00998715 -0.1664424   0.33760765  0.27392501
 -0.07771053 -0.09782731  0.00946667 -0.00232289]
Citeration_additive_adjoint: k= 1, i= 1
[-0.0195923  -0.18297669 

##### Calculate the total error and effectivity ratio

In [102]:
# Adjoint solve
phi= np.linalg.solve(M.T,psi)

# Residual
r= y.T - M @ vasoln

# Error estimate= (r,\phi)
qoi_total_error_estimate= (r @ phi)[0,0]

# Compute error in QoI directly
total_error= xstore.reshape(-1) - vasoln.T
qoi_total_error= psi.reshape(-1)@total_error

# Calculate effectivity ratio
effectivity_ratio_total= qoi_total_error_estimate / qoi_total_error
print(f'QoI_total_error                   = {qoi_total_error:13.6e}')
print(f'QoI_total_error_estimate          = {qoi_total_error_estimate:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_total:13.6e}')

QoI_total_error                   =  1.278721e-02
QoI_total_error_estimate          =  1.278721e-02
Effectivity ratio                 =  1.000000e+00


#### Calculate iteration error as the difference between the total error and the discretization error

In [103]:
qoi_iteration_error= qoi_total_error-qoi_discretization_error
print(f'QoI_iteration_error               = {qoi_iteration_error:13.6e}')

QoI_iteration_error               =  1.687916e-02


#### D iteration with error

In [104]:
# Diteration with error as a check on the accumulation of error
print('Perform D iteration with error')

Perform D iteration with error


In [105]:
w_store,wa_store,wr_store= Diteration_additive_approx(D,g,alpha,n,p,K,wpert,mpert,iprint);
print(f'norm(w_store-vglobal_store)= {np.linalg.norm(w_store-vglobal_store):13.6e}')

norm(w_store-vglobal_store)=  3.203373e-17


In [106]:
print(f'norm(wa_store-vaglobal_store)= {np.linalg.norm(wa_store-vaglobal_store):13.6e}')

norm(wa_store-vaglobal_store)=  4.320297e-17


In [107]:
print(f'norm(wr_store-rglobal_store)= {np.linalg.norm(wr_store-rglobal_store):13.6e}')

norm(wr_store-rglobal_store)=  4.131593e-17


### U iteration with error

In [108]:
print('Solving U system with error')

Solving U system with error


In [109]:
zexact,zapprox,zresid= Uhsolve_additive_approx(U,h,n,p,K,alpha,wpert,mpert,iprint)

In [110]:
if iprint >= 4:
    print('zexact')
    print(zexact.T)
    print('zapprox')
    print(zapprox.T)
    print('zresid')
    print(zresid.T)

zexact
[[ 0.01406321  0.04623057  0.07963662  0.02262606  0.04197967  0.0282514
  -0.0093479   0.08270677  0.02634515  0.04529282  0.01767509  0.03881777
   0.06914326  0.01140214  0.05996826  0.038255   -0.01861866  0.07756918
   0.03334546  0.06282098  0.01903506  0.03889105  0.06554565  0.01078518
   0.06905644  0.04282024 -0.01849901  0.07660322  0.03678929  0.07193951]]
zapprox
[[ 2.70049389e-02  2.69913008e-02  7.32749101e-02  4.62872078e-03
   6.42675395e-02 -1.27268260e-02 -6.02606606e-03  5.93025833e-02
   4.38768235e-02  6.13512623e-02 -3.22668405e-03  3.39030082e-02
   5.69761641e-02  3.06140174e-02  6.64289253e-02  1.67735413e-02
  -6.54220890e-05  1.31360668e-01  6.15596292e-02  7.14995746e-02
   1.05984996e-02  5.41317009e-02  7.61549347e-02  5.83901529e-03
   6.78307206e-02  3.33547217e-02 -2.16255573e-02  5.51200966e-02
   4.03594818e-02  9.46298164e-02]]
zresid
[[-0.01294173  0.01923927  0.00636171  0.01799734 -0.02228787  0.04097822
  -0.00332183  0.02340419 -0.017531

In [111]:
if iprint >= 2:
    print(f'Norm of difference between zexact   and vglobal_store  {np.linalg.norm(zexact-vglobal_store):13.6e}')
    print(f'Norm of difference between zapprox  and vaglobal_store {np.linalg.norm(zapprox-vaglobal_store[n:]):13.6e}')
    print(f'Norm of difference between zresid   and rglobal_store  {np.linalg.norm(zresid-rglobal_store):13.6e}')

Norm of difference between zexact   and vglobal_store   3.327777e-17
Norm of difference between zapprox  and vaglobal_store  4.550431e-02
Norm of difference between zresid   and rglobal_store   3.748869e-02


#### Calculate the discretization error and effectivity ratio

In [112]:
# Adjoint data (Adjoint system is of size nK)
psi_nK= np.zeros((n*K,1))
psi_nK[n * (K-1):n * K,0:1]= psi

# Solve adjoint equation
phi_nK= np.linalg.solve(U.T,psi_nK)
if iprint >= 2:
    print(f'Norm of difference between phi_nK and phiglobal_store {np.linalg.norm(phi_nK-phiglobal_store):13.6e}')

Norm of difference between phi_nK and phiglobal_store  2.652723e-16


In [113]:
# Compute error estimate
qoi_discretization_error_estimate= (phi_nK.T @ zresid)[0,0]

# Compute error and error in QoI directly
zsoln= zexact[-n:,0:1]
zasoln= zapprox[-n:,0:1]
zerror= zsoln - zasoln
qoi_discretization_error= (psi.T @ zerror)[0,0]

#### Calculate the effectivity ratio

In [114]:
effectivity_ratio_disc= qoi_discretization_error_estimate / qoi_discretization_error
print(f'QoI_discretization_error          = {qoi_discretization_error:13.6e}' )
print(f'QoI_discretization_error_estimate = {qoi_discretization_error_estimate:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_disc:13.3f}')

QoI_discretization_error          = -3.426811e-03
QoI_discretization_error_estimate = -3.426811e-03
Effectivity ratio                 =         1.000


#### Calculate the total error and effectivity ratio

In [115]:
# Residual
r= y - M @ zasoln

# Error estimate= (r,\phi)
qoi_total_error_estimate= r.T @ phi

# Compute error in QoI directly
total_error= xstore - zasoln
qoi_total_error= psi.T @ total_error

#Effectivity ratio
effectivity_ratio_total= qoi_total_error_estimate / qoi_total_error
print(f'QoI_total_error                   = {qoi_total_error[0,0]:13.6e}')
print(f'QoI_total_error_estimate          = {qoi_total_error_estimate[0,0]:13.6e}')
print(f'Effectivity ratio                 = {effectivity_ratio_total[0,0]:13.3f}')

QoI_total_error                   =  1.345234e-02
QoI_total_error_estimate          =  1.345234e-02
Effectivity ratio                 =         1.000


#### Calculate the iteration error as the difference between the total error and the discretization error

In [116]:
qoi_iteration_error= qoi_total_error - qoi_discretization_error
print(f'QoI_iteration_error               = {qoi_iteration_error[0,0]:13.6e}')

QoI_iteration_error               =  1.687916e-02
